In [207]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [208]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import iplot, init_notebook_mode
from matplotlib import animation,rc
from mpl_toolkits.basemap import Basemap
import base64
import io

In [209]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets, suppress_callback_exceptions=True)


In [210]:
terror=pd.read_csv('C:/Users/S145/Desktop/Teror/globalterrorismdb_0718dist.csv',encoding='ISO-8859-1')
terror.rename(columns={'iyear':'Year','imonth':'Month','iday':'Day','country_txt':'Country','region_txt':'Region','attacktype1_txt':'AttackType','target1':'Target','nkill':'Killed','nwound':'Wounded','summary':'Summary','gname':'Group','targtype1_txt':'Target_type','weaptype1_txt':'Weapon_type','motive':'Motive'},inplace=True)
terror=terror[['Year','Month','Day','Country','Region','city','latitude','longitude','AttackType','Killed','Wounded','Target','Summary','Group','Target_type','Weapon_type','Motive']]
terror['casualities']=terror['Killed']+terror['Wounded']
terror.head()

C:\Users\S145\AppData\Local\Temp\ipykernel_6220\2712116420.py:1: DtypeWarning:

Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.



,Year,Month,Day,Country,Region,city,latitude,longitude,AttackType,Killed,Wounded,Target,Summary,Group,Target_type,Weapon_type,Motive,casualities
0,1970,7,2,Dominican Republic,Central America & Caribbean,Santo Domingo,18.456792,-69.951164,Assassination,1.0,0.0,Julio Guzman,NaN,MANO-D,Private Citizens & Property,Unknown,NaN,1.0
1,1970,0,0,Mexico,North America,Mexico city,19.371887,-99.086624,Hostage Taking (Kidnapping),0.0,0.0,"Nadine Chaval, daughter",NaN,23rd of September Communist League,Government (Diplomatic),Unknown,NaN,0.0
2,1970,1,0,Philippines,Southeast Asia,Unknown,15.478598,120.599741,Assassination,1.0,0.0,Employee,NaN,Unknown,Journalists & Media,Unknown,NaN,1.0
3,1970,1,0,Greece,Western Europe,Athens,37.997490,23.762728,Bombing/Explosion,NaN,NaN,U.S. Embassy,NaN,Unknown,Government (Diplomatic),Explosives,NaN,NaN
4,1970,1,0,Japan,East Asia,Fukouka,33.580412,130.396361,Facility/Infrastructure Attack,NaN,NaN,U.S. Consulate,NaN,Unknown,Government (Diplomatic),Incendiary,NaN,NaN


In [211]:
terror.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181691 entries, 0 to 181690
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Year         181691 non-null  int64  
 1   Month        181691 non-null  int64  
 2   Day          181691 non-null  int64  
 3   Country      181691 non-null  object 
 4   Region       181691 non-null  object 
 5   city         181256 non-null  object 
 6   latitude     177135 non-null  float64
 7   longitude    177134 non-null  float64
 8   AttackType   181691 non-null  object 
 9   Killed       171378 non-null  float64
 10  Wounded      165380 non-null  float64
 11  Target       181053 non-null  object 
 12  Summary      115562 non-null  object 
 13  Group        181691 non-null  object 
 14  Target_type  181691 non-null  object 
 15  Weapon_type  181691 non-null  object 
 16  Motive       50561 non-null   object 
 17  casualities  164817 non-null  float64
dtypes: float64(5), int64(3),

In [212]:
terror.isnull().sum()

Year                0
Month               0
Day                 0
Country             0
Region              0
city              435
latitude         4556
longitude        4557
AttackType          0
Killed          10313
Wounded         16311
Target            638
Summary         66129
Group               0
Target_type         0
Weapon_type         0
Motive         131130
casualities     16874
dtype: int64

In [213]:
terror[['latitude', 'longitude','Killed', 'Wounded', 'casualities']] = terror[['latitude', 'longitude','Killed', 'Wounded', 'casualities']].fillna(terror.groupby('Country')[['latitude', 'longitude','Killed', 'Wounded', 'casualities']].transform('mean'))
terror[['city','Target']] = terror[['city','Target']].fillna(terror.groupby('Country')[['city','Target']].transform(lambda x: x.mode()[0] if not x.mode().empty else None))

In [214]:
terror.isnull().sum()

Year                0
Month               0
Day                 0
Country             0
Region              0
city                0
latitude            1
longitude           1
AttackType          0
Killed              3
Wounded             3
Target              0
Summary         66129
Group               0
Target_type         0
Weapon_type         0
Motive         131130
casualities         3
dtype: int64

In [215]:
# Calculate the sum of killed people by terrorist group
killed_people_by_terrorist = terror.groupby('Country')['Killed'].sum().reset_index()

# Get the top 10 countries with the highest number of killed people
top_n_countries = killed_people_by_terrorist.nlargest(10, 'Killed')

# Filter the top 14 terrorist groups
top_groups = terror[terror['Group'].isin(terror['Group'].value_counts()[:14].index)]

In [216]:
margins = {
    'block-margins': '10px 10px 10px 10px',
    'subblock-margins': '4px 10px 2px 4px'
}

sizes = {
    'subblock-heights': '200px'
}

color_choices = {
	'light-blue': '#7FAB8',
	'light-grey': '#F7EFED',
	'light-red':  '#F1485B',
	'dark-blue':  '#33546D',
	'middle-blue': '#61D4E2',
}

color = ['#4CCD99', '#FFC700', '#FFF455', '#007F73', '#E2F4C5']


colors = {
		'full-background':		color_choices['light-grey'],
		'chart-background':		color_choices['light-grey'],
		'histogram-color-1':	color_choices['dark-blue'],
		'histogram-color-2':	color_choices['light-red'],
		'block-borders':		color_choices['dark-blue']
}

def create_histogram(df, x_col, y_col, title, x_title, y_title, colorscale, tickangle):
    return {
        'data': [{
            'x': df[x_col].value_counts().index,
            'y': df[y_col].value_counts().values,
            'type': 'bar',
            'marker': {
                'color': df[x_col].value_counts().values,
                'colorscale': colorscale,
                'line': {'color': 'black', 'width': 1}
            }
        }],
        'layout': {
            'title': title,
            'xaxis': {
                'title': x_title,
                'tickmode': 'linear',
                'tick0': df[x_col].min(),
                'dtick': 1,
                'tickangle': tickangle
            },
            'yaxis': {'title': y_title},
            'paper_bgcolor': colors['chart-background'],
            'plot_bgcolor': colors['chart-background']
        }
    }

app.layout = html.Div([
    html.Div(children=html.H1('Terrorism Around The World'),
            style={
                'border': '3px {} solid'.format(colors['block-borders']),
                'margin': margins['block-margins'],
                'text-align': 'center'
            }
            ),

    html.Div([
        html.Div([    
            dcc.Graph(id='example_1',
                    figure={'data': [{'x': terror['Year'].value_counts().index, 'y': terror['Year'].value_counts().values, 'type': 'bar', 
                                    'marker': {
                                        'color': terror['Year'].value_counts().values,
                                        'colorscale': 'inferno',
                                        'line': {'color': 'black', 'width': 1}}
                    }],
                'layout': {
                        'title': 'Number Of Terrorist Activities Each Year',
                        'xaxis': {
                            'tickmode': 'linear',  # Ensures a continuous linear scale
                            'tick0': terror['Year'].min(),  # Start tick at min year
                            'dtick': 2,  # Tick every year
                            'tickangle': -60
                        },  
                        'yaxis': {
                            'title': 'Number of Terrorist Activities'
                        },
                        'margin': {'l': 40, 'b': 40, 't': 40, 'r': 10}, 
                        'legend':{'x': 0, 'y': 1}
                        
                            }
                            }
                        )
                            ], style = {
							'border': '1px {} solid'.format(colors['block-borders']),
							'margin': margins['block-margins'],
							'width': '48%',
							# 'height': sizes['subblock-heights'],
					}
		),
        html.Div([ 
            dcc.Graph(id='example_2',
                    figure={'data': [{'x': terror['AttackType'].value_counts().index, 'y': terror['AttackType'].value_counts().values, 'type': 'bar',
                                    'marker': {'color': terror['Year'].value_counts().values, 
                                    'colorscale': 'inferno',
                                    'line': {'color': 'black', 'width': '49%'}}
                    }],
                'layout': {
                        'title': 'Attacking Methods by Terrorists',
                        'xaxis': {
                            'tickmode': 'linear',  # Ensures a continuous linear scale
                            'tick0': terror['Year'].min(),  # Start tick at min year
                            'dtick': 1,  # Tick every year
                            'tickangle': -25
                        },
                        'yaxis': {
                            'title': 'Number of Attacking Methods'
                        },
                        'margin': {'l': 100, 'b': 120, 't': 30, 'r': 10}, 
                        'legend':{'x': 0, 'y': 1}                        
                            }
                            }
                        )
                    ], style = {
                                'border': '1px {} solid'.format(colors['block-borders']),
                                'margin': margins['block-margins'],
                                'width': '48%',
                                #'height': sizes['subblock-heights'],
                                }
		)
            ], style={'display': 'flex', 'flex-wrap': 'wrap'}),
    

        html.Div([
                html.Div([    
                    dcc.Graph(id='example',
                            figure={'data': [{'x': terror['Target_type'].value_counts().index, 'y': terror['Target_type'].value_counts().values, 'type': 'bar',
                                            'marker': {'color': terror['Target_type'].value_counts().values, 
                                            'colorscale': 'RdYlGn_r',
                                            'line': {'color': 'black', 'width': 1}}
                            }],
                        'layout': {
                                'title': 'Favorite Targets',
                                'xaxis': {
                                    'tickmode': 'linear',  # Ensures a continuous linear scale
                                    'tick0': terror['Target_type'].min(),  # Start tick at min year
                                    'dtick': 1,  # Tick every year
                                    'tickangle': -45
                                },
                                'yaxis': {
                                'title': 'Number of Favorite Targets'
                                },
                                'margin': {'l': 100, 'b': 150, 't': 35, 'r': 10}, 
                                'legend':{'x': 0, 'y': 1}         
                                    }
                                    }
                                )
                ], style = {
                                    'border': '1px {} solid'.format(colors['block-borders']),
                                    'margin': margins['block-margins'],
                                    'width': '48%',
                                    # 'height': sizes['subblock-heights'],
                                    }
                ),
            html.Div([ 
                dcc.Graph(id='example2',
                        figure={'data': [{'x': terror['Region'].value_counts().index, 'y': terror['Region'].value_counts().values, 'type': 'bar', 
                                        'marker': {'color': terror['Region'].value_counts().values, 
                                        'colorscale': 'inferno',
                                        'line': {'color': 'black', 'width': '49%'}}
                        }],
                    'layout': {
                            'title': 'Number Of Terrorist Activities By Region',
                            'xaxis': {
                                'tickmode': 'linear',  # Ensures a continuous linear scale
                                'tick0': terror['Region'].min(),  # Start tick at min year
                                'dtick': 1,  # Tick every year
                                'tickangle': -45
                            },
                            'yaxis': {
                                'title': 'Number of Activities of each Region'
                            },
                                'margin': {'l': 100, 'b': 150, 't': 35, 'r': 10}, 
                                'legend':{'x': 0, 'y': 1}   
                                }
                                }
                            )
                        ], style = {
                                    'border': '1px {} solid'.format(colors['block-borders']),
                                    'margin': margins['block-margins'],
                                    'width': '48%',
                                    #'height': sizes['subblock-heights'],
                                    }
            )
            ], style={'display': 'flex', 'flex-wrap': 'wrap'}),
        
    
    html.Div([
        html.H1("Top 10 Countries by Number of People Killed by Terrorists"),
        html.Div([
            dcc.Graph(
                id='bar-chart',
                figure=px.bar(top_n_countries, x='Country', y='Killed', color_discrete_sequence = color)
            )
        ],style = {
                                'border': '1px {} solid'.format(colors['block-borders']),
                                'margin': margins['block-margins'],
                                'width': '48%',
                                #'height': sizes['subblock-heights'],
                                } ),
        html.Div([
            dcc.Graph(
                id='pie-chart',
                figure=px.pie(top_n_countries, values='Killed', names='Country', color_discrete_sequence = color)
            )
        ], style = {
                                'border': '1px {} solid'.format(colors['block-borders']),
                                'margin': margins['block-margins'],
                                'width': '48%',
                                #'height': sizes['subblock-heights'],
                                })
        ], style={'display': 'flex', 'flex-wrap': 'wrap'}),    
    
    html.Div([
        html.Div([
            dcc.Checklist(
                id='region-checklist',
                options=[{'label': region, 'value': region} for region in terror['Region'].unique()],
                value=[terror['Region'].unique()[0]],
                labelStyle={'display': 'inline-block'}
            ),
            dcc.Graph(id='region-histogram')
        ], style={
            'border': '1px {} solid'.format(colors['block-borders']),
            'margin': margins['block-margins'],
            'width': '48%',
        }),
        html.Div([
            dcc.RadioItems(
                id='attacktype-radioitems',
                options=[{'label': attack, 'value': attack} for attack in terror['AttackType'].unique()],
                value=terror['AttackType'].unique()[0],
                labelStyle={'display': 'inline-block'}
            ),
            dcc.Graph(id='attacktype-histogram')
        ], style={
            'border': '1px {} solid'.format(colors['block-borders']),
            'margin': margins['block-margins'],
            'width': '48%',
        })
    ], style={'display': 'flex', 'flex-wrap': 'wrap'}),
    html.Div([
        html.Div([
            dcc.Dropdown(
                id='country-dropdown',
                options=[{'label': country, 'value': country} for country in terror['Country'].unique()],
                value=[terror['Country'].unique()[0]],
                multi = True
            ),
            dcc.Graph(id='country-histogram')
        ], style={
            'border': '1px {} solid'.format(colors['block-borders']),
            'margin': margins['block-margins'],
            'width': '48%',
        }),
        html.Div([
            dcc.Checklist(
                id='weapon_type-checklist',
                options=[{'label': weapon, 'value': weapon} for weapon in terror['Weapon_type'].unique()],
                value=[terror['Weapon_type'].unique()[0]],
                labelStyle={'display': 'inline-block'}
            ),
            dcc.Graph(id='weapon_type-line')
        ], style={
            'border': '1px {} solid'.format(colors['block-borders']),
            'margin': margins['block-margins'],
            'width': '48%',
        })
        ], style={'display': 'flex', 'flex-wrap': 'wrap'}),
    
    html.Div([
        html.H1("Regional Activities of Terrorist Groups"),
        dcc.Graph(
            id='map',
            figure={
                'data': [
                    go.Scattergeo(
                        lon=top_groups[top_groups['Group'] == group]['longitude'],
                        lat=top_groups[top_groups['Group'] == group]['latitude'],
                        mode='markers',
                        marker=dict(
                            size=6,
                            color=color),
                        name=group
                    )
                    for group, color in zip(top_groups['Group'].unique(), ['#4CCD99', '#FFC700', '#FFF455', '#007F73', '#E2F4C5'])
                ],
                'layout': go.Layout(
                    geo=dict(
                        showland=True,
                        landcolor='rgb(243, 243, 243)',
                        countrycolor='rgb(204, 204, 204)',
                        showcountries=True
                    ),
                    margin={'l': 40, 'b': 10, 't': 10, 'r': 5}, 
                    legend={'x': 0.8, 'y': 0.8},                    
                )
            }, style={'border': '1px {} solid'.format(colors['block-borders']), 'margin': margins['block-margins'], 'width': '200%'}
            )
            ], style={'display': 'flex', 'flex-wrap': 'wrap'}),
    ])

@app.callback( 
    Output(component_id='region-histogram', component_property='figure'), 
    [Input(component_id='region-checklist', component_property='value')] 
) 
def update_region_histogram(selected_regions): 
    traces = [] 
    for region in selected_regions: 
        filtered_df = terror[terror['Region'] == region] 
        traces.append(go.Histogram(x=filtered_df['Year'], 
                                name=region, 
                                opacity=0.75)) 
    return { 
        'data': traces, 
        'layout': go.Layout( 
            barmode='overlay', 
            yaxis={'title': 'Number of Attacks'},
            autosize=False, 
            paper_bgcolor=colors['chart-background'], 
            plot_bgcolor=colors['chart-background'], 
            margin={'l': 60, 'b': 40, 't': 20, 'r': 0}, 
            legend={'x': 0, 'y': 1}, 
        ) 
    } 

@app.callback( 
    Output(component_id='attacktype-histogram', component_property='figure'), 
    [Input(component_id='attacktype-radioitems', component_property='value')] 
) 
def update_attacktype_histogram(selected_attack): 
    filtered_df = terror[terror['AttackType'] == selected_attack] 
    traces = [ 
        go.Histogram(x=filtered_df['Year'], 
                    name=selected_attack, 
                    opacity=0.75) 
    ] 
    return { 
        'data': traces, 
        'layout': go.Layout( 
            barmode='overlay', 
            yaxis={'title': 'Number of Attacks'}, 
            autosize=False, 
            paper_bgcolor=colors['chart-background'], 
            plot_bgcolor=colors['chart-background'], 
            margin={'l': 60, 'b': 40, 't': 20, 'r': 0}, 
            legend={'x': 0, 'y': 1}, 
        ) 
    } 

@app.callback( 
    Output('country-output-container', 'children'), 
    [Input('country-dropdown', 'value')] 
) 
def update_country_checklist_container(selected_groups): 
    if not selected_groups: 
        return None 
    options = [] 
    for group in selected_groups: 
        options.extend([{'label': country, 'value': country} for country in terror['Country']]) 
    return dcc.Checklist( 
        id='country-checklist', 
        options=options, 
        value=[],  # Default no selection 
        labelStyle={'display': 'inline-block'} 
    ) 

@app.callback( 
    Output(component_id='country-histogram', component_property='figure'), 
    [Input(component_id='country-dropdown', component_property='value')] 
) 
def update_country_histogram(selected_countries): 
    traces = [] 
    for country in selected_countries: 
        filtered_df = terror[terror['Country'] == country] 
        traces.append(go.Histogram(x=filtered_df['Year'], 
                                name=country, 
                                opacity=0.75)) 
    return { 
        'data': traces, 
        'layout': go.Layout( 
            barmode='overlay', 
            yaxis={'title': 'Number of Attacks'}, 
            autosize=False, 
            paper_bgcolor=colors['chart-background'], 
            plot_bgcolor=colors['chart-background'], 
            margin={'l': 60, 'b': 40, 't': 20, 'r': 0}, 
            legend={'x': 0, 'y': 1}, 
        ) 
    } 


@app.callback( 
    Output(component_id='weapon_type-line', component_property='figure'), 
    [Input(component_id='weapon_type-checklist', component_property='value')] 
) 
def update_weapon_type_line(selected_weapons): 
    traces = [] 
    for weapon in selected_weapons: 
        filtered_df = terror[terror['Weapon_type'] == weapon] 
        grouped_df = filtered_df.groupby('Year').size().reset_index(name='count') 
        traces.append(go.Scatter(x=grouped_df['Year'], 
                                y=grouped_df['count'], 
                                mode='lines', 
                                name=weapon)) 
    return { 
        'data': traces, 
        'layout': go.Layout( 
            yaxis={'title': 'Number of Attacks'}, 
            autosize=False, 
            paper_bgcolor=colors['chart-background'], 
            plot_bgcolor=colors['chart-background'], 
            margin={'l': 60, 'b': 40, 't': 20, 'r': 0}, 
            legend={'x': 0, 'y': 1}, 
        ) 
    }


In [217]:
if __name__ == '__main__':
    app.run_server(port = 4051)